# Setup

In [ ]:
%pip install spuco

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 18.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.2/126.2 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 4.1 MB/s eta 0:00:00
  Created wheel for grad-cam: filename=grad_cam-1.5.4-py3-none-any.whl size=39588 sha256=77cce4850e117fc883b95ca6d6fbcf6201dd246fd8addb4f54bba25745b8f8c7
  Stored in directory: /root/.cache/pip/wheels/50/b0/82/1f97b5348c7fe9f0ce0ba18497202cafa5dec4562bd5292680
Successfully built grad-cam


In [ ]:
import torch

device = torch.device("cpu")

In [ ]:
from spuco.utils import set_seed

set_seed(0)

# Dataset Creation

In [ ]:
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as T

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_LARGE

trainset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.995,
    classes=classes,
    split="train",
    label_noise=0.001
)
trainset.initialize()

testset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="test"
)
testset.initialize()


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:00<00:00, 50.4MB/s]


Extracting /data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to /data/mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 1.99MB/s]

Extracting /data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to /data/mnist/MNIST/raw



Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1.65M/1.65M [00:00<00:00, 14.0MB/s]


Extracting /data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to /data/mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 4.22MB/s]

Extracting /data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to /data/mnist/MNIST/raw



In [ ]:
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]

valset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="val"
)
valset.initialize()


In [ ]:
T.ToPILImage()(trainset[1000][0]).resize((28,28))

In [ ]:
from spuco.models import model_factory
from spuco.robust_train import ERM
from torch.optim import SGD
import numpy as np
from spuco.robust_train import CustomSampleERM
from spuco.utils import Trainer
from spuco.group_inference import JTTInference
from spuco.robust_train import CustomSampleERM
from spuco.evaluate import Evaluator
import random
random.seed(10)

# Train using JTT

In [ ]:
#shuffle training set
print('Total trainset size: ' + str(len(trainset)))
shuffled_indices = np.arange(0, len(trainset))
random.shuffle(shuffled_indices)

Total trainset size: 48004


## Group Inference

In [ ]:
def gi_subset(num_partitions, partition_id, trainset, device, shuffled_indices, verbose=False):
  #generate subset index
  partition_len = int(len(trainset)/num_partitions)
  train_index = shuffled_indices[partition_len*partition_id:partition_len*partition_id+partition_len]
  train_index = train_index.tolist()
  #print(train_index)
  #train on subset
  infer_model = model_factory("lenet", trainset[0][0].shape, trainset.num_classes).to(device)
  gi_trainer = CustomSampleERM(
      model=infer_model,
      trainset=trainset,
      batch_size=64,
      optimizer=SGD(infer_model.parameters(), lr=1e-2, momentum=0.9, nesterov=True),
      num_epochs=2,
      indices=train_index,
      device=device,
      verbose=verbose
  )
  gi_trainer.train()
  #get labels
  gi_trainer_results = Trainer(
    model=infer_model,
    trainset=trainset,
    batch_size=64,
    optimizer=SGD(infer_model.parameters(), lr=1e-2, momentum=0.9, nesterov=True),
    device=device,
    verbose=verbose
  )
  output = gi_trainer_results.get_trainset_outputs()
  max_output = torch.max(output, dim=1)
  pred_labels = max_output.indices
  pred_labels = pred_labels.tolist()
  actual_labels = trainset.labels
  #jtt inference on subset
  jtt = JTTInference([pred_labels[i] for i in train_index], [actual_labels[i] for i in train_index])
  gi = jtt.infer_groups()
  nominal_upsample = len(gi[(0, 0)])/len(gi[(0, 1)])
  if verbose:
    print('\nNominal Upsample: ' + str(nominal_upsample))
    print('Majority class size: ' + str(len(gi[(0, 0)])))
    print('Minority class size: ' + str(len(gi[(0, 1)])))
  #return majority, minority group indices
  majority = [train_index[i] for i in gi[(0, 0)]]
  minority = [train_index[i] for i in gi[(0, 1)]]
  return majority, minority

In [ ]:
def full_gi(num_partitions, trainset, device, shuffled_indices, verbose=False):
  maj, min = gi_subset(num_partitions, 0, trainset, device, shuffled_indices, verbose)
  for i in range(1, num_partitions):
    maj_temp, min_temp = gi_subset(num_partitions, i, trainset, device, shuffled_indices, verbose)
    maj.extend(maj_temp)
    min.extend(min_temp)
  print('Majority class size: ' + str(len(maj)))
  print('Minority class size: ' + str(len(min)))
  nominal_upsample = int(0.8*len(maj)/len(min))
  upsampled_indices = [i for i in min for x in range(nominal_upsample)]
  upsampled_indices.extend(maj)
  print('Upsampled length: ' + str(len(upsampled_indices)))
  return upsampled_indices

In [ ]:
#upsampled_indices = full_gi(2, trainset, device, shuffled_indices, verbose=True)

## Robust Training

In [ ]:
def robust_train(trainset, testset, device, upsampled_indices, verbose=False):
  #robust training
  robust_model = model_factory("lenet", trainset[0][0].shape, trainset.num_classes).to(device)
  rt_erm = CustomSampleERM(
      model=robust_model,
      trainset=trainset,
      batch_size=64,
      optimizer=SGD(robust_model.parameters(), lr=1e-2, momentum=0.9, nesterov=True),
      num_epochs=3,
      indices=upsampled_indices,
      device=device,
      verbose=verbose
  )
  rt_erm.train()
  #evaluate robust model
  rt_evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=robust_model,
    device=device,
    verbose=verbose
  )
  rt_evaluator.evaluate()
  wg_acc = rt_evaluator.worst_group_accuracy
  avg_acc = rt_evaluator.average_accuracy
  spur_acc = rt_evaluator.evaluate_spurious_attribute_prediction()
  print('\nWorst group accuracy: ' + str(wg_acc))
  print('Average accuracy: ' + str(avg_acc))
  print('Spurious attribute accuracy: ' + str(spur_acc))
  return wg_acc, avg_acc, spur_acc

In [ ]:
#robust_train(trainset, testset, device, upsampled_indices)

## Sweep Num Partitions

In [ ]:
shuffled_indices = np.arange(0, len(trainset))
random.shuffle(shuffled_indices)
upsampled_indices = full_gi(32, trainset, device, shuffled_indices, verbose=True)
wg, avg, spur = robust_train(trainset, testset, device, upsampled_indices)

Epoch 0:  21%|██        | 5/24 [00:00<00:01, 17.37batch/s, accuracy=26.5625%, loss=1.61]

 | Epoch 0 | Loss: 1.612938642501831 | Accuracy: 15.625%
 | Epoch 0 | Loss: 1.6040290594100952 | Accuracy: 18.75%
 | Epoch 0 | Loss: 1.6284712553024292 | Accuracy: 28.125%
 | Epoch 0 | Loss: 1.6207033395767212 | Accuracy: 14.0625%
 | Epoch 0 | Loss: 1.6069189310073853 | Accuracy: 29.6875%
 | Epoch 0 | Loss: 1.6242562532424927 | Accuracy: 17.1875%
 | Epoch 0 | Loss: 1.6047077178955078 | Accuracy: 29.6875%
 | Epoch 0 | Loss: 1.6203603744506836 | Accuracy: 25.0%
 | Epoch 0 | Loss: 1.6142076253890991 | Accuracy: 26.5625%


Epoch 0:  58%|█████▊    | 14/24 [00:00<00:00, 31.09batch/s, accuracy=60.9375%, loss=1.59]

 | Epoch 0 | Loss: 1.6173579692840576 | Accuracy: 29.6875%
 | Epoch 0 | Loss: 1.599303126335144 | Accuracy: 32.8125%
 | Epoch 0 | Loss: 1.5987379550933838 | Accuracy: 25.0%
 | Epoch 0 | Loss: 1.6033940315246582 | Accuracy: 20.3125%
 | Epoch 0 | Loss: 1.611541509628296 | Accuracy: 26.5625%
 | Epoch 0 | Loss: 1.6046650409698486 | Accuracy: 26.5625%
 | Epoch 0 | Loss: 1.592217206954956 | Accuracy: 54.6875%
 | Epoch 0 | Loss: 1.5954666137695312 | Accuracy: 60.9375%
 | Epoch 0 | Loss: 1.5949854850769043 | Accuracy: 60.9375%


Epoch 0: 100%|██████████| 24/24 [00:00<00:00, 39.21batch/s, accuracy=50.0%, loss=1.58]

...

 | Epoch 1 | Loss: 1.254688024520874 | Accuracy: 73.4375%
 | Epoch 1 | Loss: 1.1208382844924927 | Accuracy: 71.42857142857143%


Getting model outputs: 100%|██████████| 751/751 [00:11<00:00, 67.30batch/s]


Nominal Upsample: 6.211538461538462
Majority class size: 1292
Minority class size: 208
Majority class size: 36279
Minority class size: 11721
Upsampled length: 71442



Evaluating group-wise accuracy: 100%|██████████| 25/25 [00:11<00:00,  2.24it/s]



Worst group accuracy: ((0, 1), 0.0)
Average accuracy: 99.39482373110386
Spurious attribute accuracy: 20.27
